In [3]:
import os
import sys
from pathlib import Path
from Bio import SeqIO
import random

# Add project root to Python path
project_root = Path.cwd().parent  
sys.path.append(str(project_root))

PRETRAIN_DATA_DIR = project_root / "data" / "raw" / "pretrain"
TRAIN_DATA_DIR = project_root / "data" / "raw" / "train"
PROCESSED_DIR = project_root / "data" / "processed"

PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

pretrain_fasta_files = [
    PRETRAIN_DATA_DIR / "bacteria_uniref.fasta",
    PRETRAIN_DATA_DIR / "archaea_uniref.fasta",
    PRETRAIN_DATA_DIR / "plants_uniref.fasta",
    PRETRAIN_DATA_DIR / "mammals_uniref.fasta",
    PRETRAIN_DATA_DIR / "fungi_uniref.fasta",
    PRETRAIN_DATA_DIR / "arthropods_uniref.fasta"
]

def combine_fasta_files_with_metadata(fasta_files, output_fasta, output_metadata):
    """Combine FASTA files and create metadata tracking file"""
    print("Step 1: Combining FASTA files with metadata tracking...")
    
    combined_sequences = []  
    metadata_records = []    
    
    for fasta_file in fasta_files:
        if fasta_file.exists():  
            source_name = fasta_file.stem.replace("_uniref", "").replace("_", " ").title()
            print(f"  Reading {source_name} from {fasta_file.name}...")
            
            count = 0  
            
            # SeqIO.parse reads FASTA files one record at a time
            for record in SeqIO.parse(fasta_file, "fasta"):
                
                # 1. Store sequence in FASTA format
                combined_sequences.append(f">{record.id}\n{str(record.seq)}")
                
                # 2. Store metadata
                metadata_records.append({
                    'sequence_id': record.id,  # Protein ID
                    'source_file': fasta_file.name,  # Which file it came from
                    'source_organism': source_name,  # Organism group
                    'sequence_length': len(record.seq),  # Length of protein
                    'sequence': str(record.seq)[:100] + "..." if len(record.seq) > 100 else str(record.seq)
                    # Store first 100 characters (for verification)
                })
                count += 1  
            
            print(f"    Added {count:,} sequences from {source_name}")
    
    print("  Shuffling sequences...")
    # Combine sequences and metadata into pairs
    combined = list(zip(combined_sequences, metadata_records))
    random.shuffle(combined)  # Randomly shuffle the pairs
    # Split them back into separate lists
    combined_sequences, metadata_records = zip(*combined)
    
    with open(output_fasta, "w") as f:
        # Join all FASTA entries with newlines and write to file
        f.write("\n".join(combined_sequences))
    
    import pandas as pd  
    df_metadata = pd.DataFrame(metadata_records)  # Convert list of dicts to DataFrame
    df_metadata.to_csv(output_metadata, index=False)  # Save as CSV
    
    print(f"\n✅ Successfully combined {len(combined_sequences):,} sequences")
    print(f"📁 Combined FASTA saved to: {output_fasta}")
    print(f"📊 Metadata saved to: {output_metadata}")
    
    return len(combined_sequences), output_fasta, output_metadata

# Define output file paths
combined_pretrain_fasta = PROCESSED_DIR / "combined_pretrain.fasta"
combined_pretrain_metadata = PROCESSED_DIR / "pretrain_metadata.csv"

# Call the function and capture the return values
sequence_count, combined_file, metadata_file = combine_fasta_files_with_metadata(
    pretrain_fasta_files,
    combined_pretrain_fasta,
    combined_pretrain_metadata
)

print("\n📈 Dataset Statistics:")
print(f"Total sequences: {sequence_count:,}")  # Use the returned count
print(f"Files combined: {len([f for f in pretrain_fasta_files if f.exists()])}")

# Check organism distribution (need to read the metadata CSV)
import pandas as pd  # Import pandas if not already imported
df_meta = pd.read_csv(metadata_file)  # Read the metadata CSV we just created

print("\n📊 Distribution by organism:")
print(df_meta['source_organism'].value_counts())

# Length statistics
print(f"\n📏 Sequence length statistics:")
print(f"Min length: {df_meta['sequence_length'].min()}")
print(f"Max length: {df_meta['sequence_length'].max()}")
print(f"Average length: {df_meta['sequence_length'].mean():.1f}")

# Show file sizes
import os
print(f"\n💾 File sizes:")
print(f"Combined FASTA: {os.path.getsize(combined_file) / (1024**3):.2f} GB")
print(f"Metadata CSV: {os.path.getsize(metadata_file) / (1024**2):.2f} MB")

# Show sample of the data
print(f"\n👀 Sample entries (first 3):")
print(df_meta.head(3).to_string())

Step 1: Combining FASTA files with metadata tracking...
  Reading Bacteria from bacteria_uniref.fasta...
    Added 243,465 sequences from Bacteria
  Reading Archaea from archaea_uniref.fasta...
    Added 815,399 sequences from Archaea
  Reading Plants from plants_uniref.fasta...
    Added 826,491 sequences from Plants
  Reading Mammals from mammals_uniref.fasta...
    Added 910,034 sequences from Mammals
  Reading Fungi from fungi_uniref.fasta...
    Added 985,183 sequences from Fungi
  Reading Arthropods from arthropods_uniref.fasta...
    Added 1,880,722 sequences from Arthropods
  Shuffling sequences...

✅ Successfully combined 5,661,294 sequences
📁 Combined FASTA saved to: c:\Users\USER\Documents\cod3astro\ML_AI\ProteinSeq_DL\data\processed\combined_pretrain.fasta
📊 Metadata saved to: c:\Users\USER\Documents\cod3astro\ML_AI\ProteinSeq_DL\data\processed\pretrain_metadata.csv

📈 Dataset Statistics:
Total sequences: 5,661,294
Files combined: 6

📊 Distribution by organism:
source_organ